In [2]:
import json
import time
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cv2

In [3]:
# Opening JSON file
start = time.time()
with open('dict_spottings.json') as json_file:
    data = json.load(json_file)

In [4]:
videoLocation = 'D:/BOBSL/BOBSL200G/bobsl/videos'
videoTitles = os.listdir(videoLocation)

In [6]:
videoName = videoTitles[0]
print('opening: ' + videoLocation + videoName)
targetVideo = videoLocation + videoName

opening: D:/BOBSL/BOBSL200G/bobsl/videos5087953980081062580.mp4
